# Factor Expressions

In [1]:
# Large volume implies bullish

# group_neutralize(
#   volume / (ts_sum(volume, 60) / 60),
#   sector
# )

'''
ts_sum: sum over last 60 days, 
group_neutralize: an int operator, neutralize alpha over the group, can use customized group for neutralization
'''

# Improvement

# ts_step(20) * volume / (ts_sum(volume, 60) / 60)
'''
ts_step: n for today, n-1 for yesterday, ... 1 for last day; Smooth over the last 20 days, control the turnover
'''

'\nts_step: n for today, n-1 for yesterday, ... 1 for last day; Smooth over the last 20 days, control the turnover\n'

In [2]:
# Reversion

# rank(
#   -(close - (ts_product(close, 5)) ^ 0.2)
# )

'''
ts_product: product over last n days
rank: weight equally between 0~1
'''

# Improvement

# rank( -signedpower(close - sum(close, 5)/5, 2))

'''
signedpower(x, e) = sign(x) * (abs(x) ^ e)
'''

In [ ]:
# Idea: if volume ratio is increasing, then we expect a reversal because the market is absorbing the price info
# if the volume ratio is decreasing, then the price will continue the trend because the market hasn't absorbed the info yet 


# - ts_corr(
#   rank(close), rank(volume/adv20), 5
#)
'''
ts_corr: correlation over last n days
'''

# Possible improvement: compare short corr vs long corr 

In [ ]:
# Combine ideas

# A = expr1
# B = expr2
# rank(scale(A, scale=1, longscale=1, shortscale=1) + scale(B, scale=1, longscale=1, shortscale=1))

In [ ]:
# GroupMean
# rank(
#   group_mean(ts_delta(close, 5), 1, subindustry) - ts_delta(close, 5))
#)

In [3]:
# Earningn per share

# Looks like the alpha won't work
# (rank(eps/last_diff_value(eps, 5)) > 0.7 || volume > ts_delay(volume, 1) ?
# rank(-ts_delta(close, 5)) : -1

'''
last_diff_value: diff for non-frequent data change
'''

'\nlast_diff_value: diff for non-frequent data change\n'

In [ ]:
# 